# 라이브러리

In [50]:
import numpy as np

from bs4 import BeautifulSoup

import xmltodict

from PIL import Image

import json

from tqdm import tqdm

import pandas as pd

import pickle

import cv2

import os

import glob

# import tensorflow as tf

# import keras

# import matplotlib.pyplot as plt

# from tensorflow.keras import datasets, layers, models

from sklearn.model_selection import train_test_split

# 좌표변환

In [46]:
import xml.etree.ElementTree as ET
import glob
import os
import json


def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1] + 840) / 2) / (h + 840)
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / (h + 840)
    return [x_center, y_center, width, height]


def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]


classes = []
# input_dir = "annotations/"
# output_dir = "labels/"
# image_dir = "images/"
input_dir = "Astrophysics_xml\\"
output_dir = "labels\\"
image_dir = "Astrophysics_image\\"


# create the labels folder (output directory)
# os.mkdir(output_dir)

# identify all the xml files in the annotations folder (input directory)
files = glob.glob(os.path.join(input_dir, '*.xml'))
# loop through each 
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    # check if the label contains the corresponding image file

    tree = ET.parse(fil)
    root = tree.getroot()
    Str = root.find('path').text
    menu_info = Str.split('\\')

    if not os.path.exists(os.path.join(image_dir, menu_info[-3], menu_info[-2], f"{filename}.png")):
        print(f"{filename} image does not exist!")
    
        continue
    
    result = []

    # parse the content of the xml file

    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall('object'):
        label = obj.find("name").text
        # check for new classes and append to list
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        # generate a YOLO format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

# generate the classes file as reference
with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))

H_8543.70-4010_01_051 image does not exist!
H_8543.70-4010_01_052 image does not exist!
H_8543.70-4010_01_053 image does not exist!
H_8543.70-4010_01_054 image does not exist!
H_8543.70-4010_01_055 image does not exist!
H_8543.70-4010_01_056 image does not exist!
H_8543.70-4010_01_057 image does not exist!
H_8543.70-4010_01_058 image does not exist!
H_8543.70-4010_01_059 image does not exist!
H_8543.70-4010_01_060 image does not exist!
H_8543.70-4010_01_061 image does not exist!
H_8543.70-4010_01_062 image does not exist!
H_8543.70-4010_01_063 image does not exist!
H_8543.70-4010_01_064 image does not exist!
H_8543.70-4010_01_065 image does not exist!
H_8543.70-4010_01_066 image does not exist!
H_8543.70-4010_01_067 image does not exist!
H_8543.70-4010_01_068 image does not exist!
H_8543.70-4010_01_069 image does not exist!
H_8543.70-4010_01_070 image does not exist!
H_8543.70-4010_01_071 image does not exist!
H_8543.70-4010_01_072 image does not exist!
H_8543.70-4010_01_073 image does

# resize

In [52]:
input_dir = "Astrophysics_xml\\"
output_dir = "labels\\"
image_dir = "Astrophysics_image\\"

files = glob.glob(os.path.join(input_dir, '*.xml'))
# loop through each 
for fil in tqdm(files):
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    # check if the label contains the corresponding image file

    tree = ET.parse(fil)
    root = tree.getroot()
    Str = root.find('path').text
    menu_info = Str.split('\\')

    if not os.path.exists(os.path.join(image_dir, menu_info[-3], menu_info[-2], f"{filename}.png")):
        print(f"{filename} image does not exist!")
    
        continue
    
    file_dir = os.path.join(image_dir, menu_info[-3], menu_info[-2], f"{filename}.png")
    
    want_size = 480    #원하는 크기의 정사각형 한변의 길이
    
    img = cv2.imread(file_dir)

    percent = 1
    if(img.shape[1] > img.shape[0]) :
        percent = want_size/img.shape[1]
    else :
        percent = want_size/img.shape[0]

    img = cv2.resize(img, dsize=(0, 0), fx=percent, fy=percent, interpolation=cv2.INTER_LINEAR)

    y,x,h,w = (0,0,img.shape[0], img.shape[1])

    w_x = (want_size-(w-x))/2
    h_y = (want_size-(h-y))/2

    if(w_x < 0):
        w_x = 0
    elif(h_y < 0):
        h_y = 0

    M = np.float32([[1,0,w_x], [0,1,h_y]])
    img_re = cv2.warpAffine(img, M, (want_size, want_size))
    
    cv2.imwrite(f'C:/Users/chois/Downloads/TeamData/Astrophysics_resize_image/{filename}.jpg', img_re)

  3%|▎         | 4784/137708 [05:11<2:42:31, 13.63it/s]